In [1]:
import numpy as np

In [2]:
# 載入資料
X_train = np.load('X_train.npy')
X_val = np.load('X_val.npy')
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy')
y_val = np.load('y_val.npy')
y_test = np.load('y_test.npy')

In [9]:
# 重塑數據形狀為 (樣本數, 高度, 寬度, 頻道數)
X_train = X_train[..., np.newaxis]
X_val = X_val[..., np.newaxis]
X_test = X_test[..., np.newaxis]

X_train = X_train / np.max(X_train)
X_val = X_val / np.max(X_val)
X_test = X_test / np.max(X_test)

# print(X_train.shape)
# print(X_val.shape)
# print(X_test.shape)



[[[[[nan]]]


  [[[nan]]]


  [[[nan]]]


  ...


  [[[nan]]]


  [[[nan]]]


  [[[nan]]]]



 [[[[nan]]]


  [[[nan]]]


  [[[nan]]]


  ...


  [[[nan]]]


  [[[nan]]]


  [[[nan]]]]



 [[[[nan]]]


  [[[nan]]]


  [[[nan]]]


  ...


  [[[nan]]]


  [[[nan]]]


  [[[nan]]]]



 ...



 [[[[nan]]]


  [[[nan]]]


  [[[nan]]]


  ...


  [[[nan]]]


  [[[nan]]]


  [[[nan]]]]



 [[[[nan]]]


  [[[nan]]]


  [[[nan]]]


  ...


  [[[nan]]]


  [[[nan]]]


  [[[nan]]]]



 [[[[nan]]]


  [[[nan]]]


  [[[nan]]]


  ...


  [[[nan]]]


  [[[nan]]]


  [[[nan]]]]]


In [12]:
print(X_train[15])

[[[[[nan]]]


  [[[nan]]]


  [[[nan]]]


  ...


  [[[nan]]]


  [[[nan]]]


  [[[nan]]]]



 [[[[nan]]]


  [[[nan]]]


  [[[nan]]]


  ...


  [[[nan]]]


  [[[nan]]]


  [[[nan]]]]



 [[[[nan]]]


  [[[nan]]]


  [[[nan]]]


  ...


  [[[nan]]]


  [[[nan]]]


  [[[nan]]]]



 ...



 [[[[nan]]]


  [[[nan]]]


  [[[nan]]]


  ...


  [[[nan]]]


  [[[nan]]]


  [[[nan]]]]



 [[[[nan]]]


  [[[nan]]]


  [[[nan]]]


  ...


  [[[nan]]]


  [[[nan]]]


  [[[nan]]]]



 [[[[nan]]]


  [[[nan]]]


  [[[nan]]]


  ...


  [[[nan]]]


  [[[nan]]]


  [[[nan]]]]]


In [4]:
import tensorflow as tf
from keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

def create_model(input_shape):
    input_img = Input(shape=input_shape)
    x = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(input_img)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=input_img, outputs=output)
    return model

In [5]:
input_shape = np.shape(X_train)[1:]
model = create_model(input_shape)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 2757, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 64, 2757, 32)      320       
                                                                 
 batch_normalization (BatchN  (None, 64, 2757, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 1379, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 1379, 32)      9248      
                                                                 
 batch_normalization_1 (Batc  (None, 32, 1379, 32)     128   

# 編譯模型

In [6]:
from keras.optimizers import Adam

initial_learning_rate = 0.01

# 創建 Adam 優化器並指定學習率
optimizer = Adam(learning_rate=initial_learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# 訓練模型

In [7]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=16)

Epoch 1/50


ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/model/batch_normalization/FusedBatchNormGradV3' defined at (most recent call last):
    File "C:\Users\User\miniconda3\envs\wind_turbine\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\User\miniconda3\envs\wind_turbine\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\User\miniconda3\envs\wind_turbine\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "C:\Users\User\miniconda3\envs\wind_turbine\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "C:\Users\User\miniconda3\envs\wind_turbine\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "C:\Users\User\miniconda3\envs\wind_turbine\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
   